
## Pré-Processing

Neste jupyter notebook, iremos realizar uma análise comparativa entre os extratores de features da face: MediaPipe e Dlib

As features foram extraídas utilizando as seguintes configurações de máquina:
    
    - 15,4 GiB
    - 11th Gen Intel® Core™ i5-1135G7 @ 2.40GHz × 8 
    - Mesa Intel® Xe Graphics (TGL GT2)
    - 256,1 GB
    - Ubuntu 20.04.3 LTS
    - 64 bits
    
Foram utilizados 591 vídeos da base UCF-101, ou seja, todos os vídeos das classes de:
    - ApplyEyeMakeup
    - BrushingTeeth
    - PlayingFlute
    - ShavingBeard
    
A escolha dessas classes de comportamentos, tem como justificativa classes de comportamento que de certa forma enfatizam a face humana durante os vídeos, utilizando movimentos, interações com objetos e cenas com oclusão da face sejam por esses objetos ou membro do corpo, ou pelo movimento associado ao comportamento.

In [2]:
import os
import glob
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#Dlib
os.chdir('../../data/processed/dlib/UFC-101-Analisys/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
print("Quantidade de Vídeos [Dlib]: "+str(len(all_filenames)))

combined_csv_dlib = pd.concat([pd.read_csv(f, sep=";", index_col=None) for f in all_filenames ])
print('Quantidade de Frames [Dlib]: '+str(combined_csv_dlib.shape[0]))
combined_csv_dlib['extractor'] = 'dlib'


#Mediapipe
os.chdir('../../mediapipe/UFC-101-Analisys/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
print("Quantidade de Vídeos [Mediapipe]: "+str(len(all_filenames)))

combined_csv_mediapipe = pd.concat([pd.read_csv(f, sep=";", index_col=None) for f in all_filenames ])
print('Quantidade de Frames [Mediapipe]: '+str(combined_csv_mediapipe.shape[0]))
combined_csv_mediapipe['extractor'] = 'mediapipe'

#os.chdir('../../../../notebooks/feature_extraction/')

In [ ]:
combined_csv_mediapipe.head(5)

In [ ]:
combined_csv_dlib.head(5)

In [ ]:
combined_csv = pd.concat([combined_csv_dlib, combined_csv_mediapipe], axis=0)
print('Linhas, Colunas: '+str(combined_csv.shape))
print('Quantidade de Frames [Dlib]: '+str(combined_csv.loc[combined_csv.extractor == 'dlib'].shape))
print('Quantidade de Frames [Mediapipe]: '+str(combined_csv.loc[combined_csv.extractor == 'mediapipe'].shape))

In [ ]:
df_geral = pd.DataFrame(columns=['extractor', 'total_notna'])

df_geral = df_geral.append(
[{
    'extractor': 'dlib',
    'total_notna': int(combined_csv.loc[combined_csv.extractor == 'dlib']['keys'].notna().sum())
},
{ 
    'extractor': 'mediapipe',
    'total_notna': int(combined_csv.loc[combined_csv.extractor == 'mediapipe']['keys'].notna().sum())
}],
    
ignore_index=True)

df_geral

In [ ]:
combined_csv['na'] = np.where(combined_csv['keys'].isna(), 1, 0)
combined_csv_dlib['na'] = np.where(combined_csv_dlib['keys'].isna(), 1, 0)
combined_csv_mediapipe['na'] = np.where(combined_csv_mediapipe['keys'].isna(), 1, 0)

In [ ]:
combined_csv

In [ ]:
print(len(combined_csv.loc[(combined_csv.extractor == 'mediapipe') & (combined_csv['keys'].notna())])/len(combined_csv))
print(len(combined_csv.loc[(combined_csv.extractor == 'mediapipe') & (combined_csv['keys'].isna())])/len(combined_csv))
print(len(combined_csv.loc[(combined_csv.extractor == 'dlib') & (combined_csv['keys'].notna())])/len(combined_csv))
print(len(combined_csv.loc[(combined_csv.extractor == 'dlib') & (combined_csv['keys'].isna())])/len(combined_csv))

# Análise

## No Geral

In [ ]:
# plot
fig = plt.figure(figsize=(10,10))
plt.subplot(2,2,1)

ax = sns.countplot(x="extractor", hue="na", data=combined_csv_dlib)
ax.set(ylabel='Total of Frames', title='Bar Count and Percent of Total')

# add annotations
for c in ax.containers:
    # custom label calculates percent and add an empty string so 0 value bars don't have a number
    labels = [f'{w/len(combined_csv_dlib.index)*100:0.1f}%' if (w := v.get_height()) > 0 else '' for v in c]
    ax.bar_label(c, labels=labels, label_type='edge')

    
plt.subplot(2,2,2)

ax = sns.countplot(x="extractor", hue="na", data=combined_csv_mediapipe)
ax.set(ylabel='Total of Frames', title='Bar Count and Percent of Total')

# add annotations
for c in ax.containers:
    # custom label calculates percent and add an empty string so 0 value bars don't have a number
    labels = [f'{w/len(combined_csv_mediapipe.index)*100:0.1f}%' if (w := v.get_height()) > 0 else '' for v in c]
    ax.bar_label(c, labels=labels, label_type='edge')

In [ ]:
82.1 - 32.1

Na figura acima é possível visualizar que dos 134.484 frames da base selecionada para a análise, apenas em 32,1% frames desse total, o Dlib foi capaz de reconhecer os landmarks da face. Enquanto que o extrator Mediapipe conseguiu detectar os landmarks em 82,1% dos frames. Portanto, de maneira geral, temos que com o extrator de features Mediapipe foi possível detectar landmarks em 50% de frames a mais que no Dlib.


## Por vídeo

In [ ]:
combined_csv['class'] = combined_csv['video_name'].str.split('_', expand=True)[1]
combined_csv.head(2)

In [ ]:
fig = plt.figure(figsize=(14,10))
count = 1

for i in combined_csv['class'].unique():
    #col, row, order
    plt.subplot(2,2, count)
    df = combined_csv.loc[combined_csv['class'] == i]
    ax = sns.countplot(x="extractor", hue="na", data=df)
    ax.set(ylabel='Total of Frames', title=i)
    for c in ax.containers:
        labels = [f'{w/len(df.index)*200:0.1f}%' if (w := v.get_height()) > 0 else '' for v in c]
        ax.bar_label(c, labels=labels, label_type='edge')
    count+=1
    

Tanto para o extrator Mediapipe quando para o Dlib, a classe em que foi retornada mais frames com informações da face foi a ApplyEyeMakeup com 47,5% e 30,5% respectivamente (diferença de 17%). Enquanto que para a classe de comportamento PlayingFlute, ambos os extratores de features retornaram os valores mais baixos de frames com features detectadas, o Mediapipe 35,4% e o Dlib 5,6% (diferença de 29,8%). 

## Por momentos específicos

In [ ]:
!sudo apt-get install ffmpeg

In [ ]:
import mediapy as media
url = '../../../../data/processed/examples/v_ShavingBeard_g03_c03-mediapipe.avi'
video = media.read_video(url)
media.show_video(video)  

In [ ]:
from IPython.display import Video

Video("../../../../data/processed/examples/v_ShavingBeard_g03_c03-mediapipe.avi", embed=True)

In [ ]:
from IPython.display import HTML


HTML("""
<video width="320" height="240" controls>
  <source src="../../../../data/processed/examples/v_ShavingBeard_g03_c03-mediapipe.avi" type="video/avi">
</video>
""")

In [ ]:
os.listdir('../../../../data/processed/examples/')

In [ ]:
combined_csv.head(1)

In [ ]:
47.5 - 30.5

In [ ]:
for i in combined_csv['class'].unique():
    print(i)